In [277]:
#!/usr/bin/env python
# Copyright (C) 2017 Sur Herrera Paredes

# Imports
import os
import sutilspy
import csv
import numpy as np
import scipy.stats as stats

In [278]:
# Arguments
indir = 'snps/'
test = 'MK'
outfile = 'mk_results.txt'
metadata_file = 'map.txt'
group1 = 'A'
group2 = 'B'
min_count = 3
nrows = 30000

In [279]:
class GenomeSite:
    """A class for represintinc sites in genome that have potential SNPS"""
    
    def __init__(self,site_id, contig, position, ref_allele = '',
                 major_allele = '',
                 minor_allele = '', locus_type = '', gene_id = '',
                 aminoacid_A = '',
                 aminoacid_C = '', aminoacid_G = '', aminoacid_T = ''):
        self.id = site_id
        self.contig = contig
        self.position = position
        self.ref_allele = ref_allele
        self.major_allele = major_allele
        self.minor_allele = minor_allele
        self.locus_type = locus_type
        self.gene_id = gene_id
        self.aminoA = aminoacid_A
        self.aminoC = aminoacid_C
        self.aminoG = aminoacid_G
        self.aminoT = aminoacid_T
    
    def codon_aminoacid(self, base):
        if base in ['A','a']:
            return(self.aminoA)
        elif base in ['C','c']:
            return(self.aminoA)
        elif base in ['G','g']:
            return(self.aminoA)
        elif base in ['T','t']:
            return(self.aminoA)
        else:
            raise ValueError("base must be one of the four canonical nucleoties")
    
    def substitution_type(self):
        substitution_type = ''
        if self.codon_aminoacid(base = self.major_allele) == self.codon_aminoacid(base = self.minor_allele):
            substitution_type = 'synonymous'
        else:
            substitution_type = 'non-synonymous'
        
        return(substitution_type)
            

In [280]:
class Gene:
    """A class for representing a gene"""
    
    def __init__(self, gene_id,contig,start,end, strand = ''):
        if(start > end):
            raise ValueError("Start cannot be greater than end")
        self.id = gene_id
        self.contig = contig
        self.start = start
        self.end = end
        self.strand = strand
    
    def extend(self, pos):
        if pos > self.end:
            self.end = pos
        elif pos < self.start:
            self.start = pos
    
    def print(self):
        print("===Gene===")
        print(">Gene id: {}".format(self.id))
        print(">Gene start: {}".format(str(self.start)))
        print(">Gene end: {}".format(str(self.end)))
            
    

In [281]:
class MKtest:
    """A class for holding the McDonald-Kreitmant test"""
    
    def __init__(self, name, Ds = 0, Dn = 0, Ps = 0, Pn = 0):
        self.name = name
        self.Dn = Dn
        self.Ds = Ds
        self.Ps = Ps
        self.Pn = Pn
    
    def update(self, Ds = 0, Dn = 0, Ps = 0, Pn = 0):
        """Update the contigency matrix"""
        self.Dn += Dn
        self.Ds += Ds
        self.Ps += Ps
        self.Pn += Pn
    
    def calculate(self):
        """Calculate the McDonald Kreitman ratio"""
        ratio = (self.Dn / self.Ds) / (self.Pn / self.Ps)
        return(ratio)
    
    def alpha(self):
        """Calculate the Smith & Eyre-Walker alpha"""
        alpha = 1 - ((self.Ds * self.Dn) / (self.Ps * self.Pn))
        return(alpha)
    
    def test(self):
        res = stats.fisher_exact([[self.Ds,self.Ps],[self.Dn,self.Pn]])
        return(res)
    

        

In [282]:
# Check files exist in input directory
file_list = os.listdir(indir)
if 'snps_freq.txt' not in file_list:
    raise FileNotFoundError("Could not find snps_freq.txt at {}".format(indir))
if 'snps_info.txt' not in file_list:
    raise FileNotFoundError("Could not find snps_info.txt at {}".format(indir))
if 'snps_depth.txt' not in file_list:
    raise FileNotFoundError("Could not find snps_depth.txt at {}".format(indir))
if not os.path.isfile(metadata_file):
    raise FileNotFoundError("Could not find metadata file {}".format(metadata_file))


In [283]:
# Read metadata
Groups = sutilspy.io.process_run_list(metadata_file, 1, 0, header = True)
Samples = sutilspy.io.process_run_list(metadata_file, 0, 1, header = True)


> Processing map of runs
	Processed 4 runs in 2 samples

> Processing map of runs
	Processed 4 runs in 4 samples


In [284]:
# Read info
Genes = {}
Sites = {}
with open(indir + '/snps_info.txt') as info_fh:
    header = info_fh.readline()
    header = header.split('\t')
    print(header)
    info_reader = csv.reader(info_fh, delimiter = '\t')
    i = 0
    
    # Set columns
    site_id_col = 0
    contig_col = 1
    pos_col = 2
    ref_allele_col = 3
    major_allele_col = 4
    minor_allele_col = 5
    locus_type_col = 11
    gene_id_col = 12
    aminoacids_col = 15
    
    print("============HEADERs============")
    print(">Site id: {}".format(header[site_id_col]))
    print(">Contig: {}".format(header[contig_col]))
    print(">Position: {}".format(header[pos_col]))
    print(">Ref allele: {}".format(header[ref_allele_col]))
    print(">Major allele: {}".format(header[major_allele_col]))
    print(">Minor allele: {}".format(header[minor_allele_col]))
    print(">Locus type: {}".format(header[locus_type_col]))
    print(">Gene id: {}".format(header[gene_id_col]))
    print(">Aminoacids: {}".format(header[aminoacids_col]))
    
    #
    for row in info_reader:
        i += 1
        #print(row)
        #print(row[gene_id_col], row[site_id_col])
        #print(row[aminoacids_col])
        gene = row[gene_id_col]
        site_id = row[site_id_col]
        aminoacids = row[aminoacids_col]
        #print(aminoacids)
        
        if gene == 'NA':
            # skip intergenig regions
            continue
        
        # Get aminoacid per position
        aa = aminoacids.split(',')
        #print(aa)
        
        # Define site
        Sites[site_id] = GenomeSite(site_id = site_id,
                                    contig = row[contig_col],
                                    position = row[pos_col],
                                    ref_allele = row[ref_allele_col],
                                    major_allele = row[major_allele_col],
                                    minor_allele = row[minor_allele_col],
                                    locus_type = row[locus_type_col],
                                    gene_id = gene, aminoacid_A = aa[0],
                                    aminoacid_C = aa[1],
                                    aminoacid_G = aa[2],
                                    aminoacid_T = aa[3])
        
        # For genes
        if gene in Genes:
            # update genes
            Genes[gene].extend(row[pos_col])
            #print(gene)
            #print(Genes[gene])
            #Genes[gene].print()

        else:
            # Define gene
            Genes[gene] = Gene(gene_id=gene, contig = row[contig_col],
                               start = row[pos_col], end = row[pos_col])
            #Genes[gene].print()
            #print(Genes[gene])
        
        if i > nrows:
            break 
info_fh.close()

['site_id', 'ref_id', 'ref_pos', 'ref_allele', 'major_allele', 'minor_allele', 'count_samples', 'count_a', 'count_c', 'count_g', 'count_t', 'locus_type', 'gene_id', 'snp_type', 'site_type', 'amino_acids\n']
============HEADERs============
>Site id: site_id
>Contig: ref_id
>Position: ref_pos
>Ref allele: ref_allele
>Major allele: major_allele
>Minor allele: minor_allele
>Locus type: locus_type
>Gene id: gene_id
>Aminoacids: amino_acids



In [286]:
#print(Groups)

In [287]:
# Chose sites based on depth in groups to compare
Counts = {}
with open(indir + '/snps_depth.txt') as depth_fh:
    header = depth_fh.readline()
    header = header.rstrip()
    header = header.split('\t')
    
    # Get sample and column indices
    samples = header[1:]
    indices = {}
    for s in samples:
        indices[s] = header.index(s)
    print(indices)
    
    
    depth_reader = csv.reader(depth_fh, delimiter = '\t')
    i = 0
    for row in depth_reader:
        i += 1
        #print(row)
        
        site_id = row[0]
        #print(site_id)
        
        # Get all counts
        counts = row[1:]
        counts = list(map(int,counts))
        #print(counts)
        
        counts = [int(c >= min_count) for c in counts]
        
        # Get counts per group
        samples1 = [int(counts[ indices[l] - 1 ]) for l in Groups[group1]]
        samples2 = [int(counts[ indices[l] - 1 ]) for l in Groups[group2]]
        samples1 = sum(samples1)
        samples2 = sum(samples2)
        #print(samples1)
        #print(samples2)
        if not ((samples1 > 0 and samples2 > 0) and (samples1 > 1 or samples2 > 1)):
            # delete
            #print(site_id)
            if site_id in Sites:
                del Sites[site_id]
        else:
            # NOTE: ASSUMING SAME ORDER IN SAMPLES BETWEEN SITES
            Counts[site_id] = counts
        
        if i > nrows:
            break 

depth_fh.close()

{'sample1': 1, 'sample2': 2, 'sample3': 3, 'sample4': 4}


In [294]:
# Read frequencies and calculate 
print(Groups)
MK = {}
with open(indir + '/snps_freq.txt') as freqs_fh:
    header = freqs_fh.readline()
    header = header.rstrip()
    header = header.split('\t')
    
    # Get sample and column indices
    samples = header[1:]
    indices = {}
    for s in samples:
        indices[s] = header.index(s)
    print(indices)
    print(header)
    
    freqs_reader = csv.reader(freqs_fh, delimiter = '\t')
    i = 0
    for row in freqs_reader:
        i += 1
        
        # Check if site was selected based on sites
        site_id = row[0]
        if not site_id in Sites:
            #print("==Skipping")
            continue
        
        #print("==========================")
        gene = Sites[site_id].gene_id
        s_type = Sites[site_id].substitution_type()
        present_index = np.array(Counts[site_id])
        group_index = np.array([Samples[s][0] for s in samples])
        #print(row)
        #print(site_id)
        #allele_frequencies = row.split()
        #print("Major Allele: {}".format(Sites[site_id].major_allele))
        #print("Minor Allele: {}".format(Sites[site_id].minor_allele))
        #print("Substitution type: {}".format(s_type))
        #print("Gene: {}".format(gene))
        #print(present_index)
        #print(group_index)
        
        # Create MKtest if needed
        if gene not in MK:
            MK[gene]= MKtest(name=gene)
            
        # find allele per sample
        allele_freqs = np.array([int(float(f) < 0.5) for f in row[1:]])
        #print(allele_freqs)
        
        # Remove non covered positions
        ii = np.where(present_index)
        group_index = group_index[ii]
        allele_freqs = allele_freqs[ii]
        #print(group_index)
        #print(allele_freqs)
        
        # Count alleles per group
        group1_count = allele_freqs[np.where(group_index == group1)].sum()
        group2_count = allele_freqs[np.where(group_index == group2)].sum()
        #print(group1_count)
        #print(group2_count)
        
        if group1_count > 0 and group2_count > 0:
            fixed = False
        elif group1_count > 0 or group2_count > 0:
            fixed = True
#         else:
#             print("===============")
#             print(row)
#             print(group_index)
#             print(allele_freqs)
#             raise ValueError("At least one of the counts must be non-zero")
        
        if s_type is 'synonymous':
            if fixed:
                MK[gene].update(Ds = 1)
            else:
                MK[gene].update(Ps = 1)
        elif s_type is 'non-synonymous':
            if fixed:
                MK[gene].update(Dn = 1)
            else:
                MK[gene].update(Pn = 1)
        else:
            raise ValueError("Invalid substitution type")
            
        #print("==========================")
        
        
        if i > nrows:
            break
freqs_fh.close()

{'A': ['sample1', 'sample3'], 'B': ['sample2', 'sample4']}
{'sample1': 1, 'sample2': 2, 'sample3': 3, 'sample4': 4}
['site_id', 'sample1', 'sample2', 'sample3', 'sample4']


In [296]:
with open(outfile,mode='w') as fh:
    for gene,mk in MK.items():
        #print(gene)
        #print("\t\tFixed\tPolymorphic\n\tSynonymous\t{}\t{}\n\tnon-synonymous\t{}\t{}".format(mk.Ds,mk.Ps,mk.Dn,mk.Pn))
        #ratio = mk.calculate()
        oddratio,pval = mk.test()
        res = [gene, str(oddratio), str(pval)]
        #print(res)
        fh.write("\t".join(res) + "\n")
        #alpha = mk.alpha()
        #print("MK ratio is: {}".format(str(ratio)))
        #print("MK alpha is: {}".format(str(alpha)))

['862965.3.peg.2', 'nan', '1.0']
['862965.3.peg.4', 'nan', '1.0']
['862965.3.peg.6', 'nan', '1.0']
['862965.3.peg.8', 'nan', '1.0']
['862965.3.peg.9', 'nan', '1.0']
['862965.3.peg.10', 'nan', '1.0']
['862965.3.peg.11', 'nan', '1.0']
['862965.3.peg.14', 'nan', '1.0']
['862965.3.peg.15', 'nan', '1.0']
['862965.3.peg.17', 'nan', '1.0']
['862965.3.peg.18', 'nan', '1.0']
['862965.3.peg.19', 'nan', '1.0']
['862965.3.peg.20', 'nan', '1.0']
['862965.3.peg.21', 'nan', '1.0']
['862965.3.peg.22', 'nan', '1.0']
['862965.3.peg.26', 'nan', '1.0']
['862965.3.peg.28', 'nan', '1.0']
['862965.3.peg.29', 'nan', '1.0']
['862965.3.peg.32', 'nan', '1.0']
['862965.3.peg.34', 'nan', '1.0']
['862965.3.peg.38', 'nan', '1.0']
['862965.3.peg.39', 'nan', '1.0']
['862965.3.peg.40', 'nan', '1.0']
['862965.3.peg.41', 'nan', '1.0']
['862965.3.peg.43', 'nan', '1.0']
['862965.3.peg.45', 'nan', '1.0']
['862965.3.peg.48', 'nan', '1.0']
['862965.3.peg.49', 'nan', '1.0']
['862965.3.peg.50', 'nan', '1.0']
['862965.3.peg.52',